<h1>Team 80 Work</h1>

In [1]:
sc

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1649088137482_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

<SparkContext master=yarn appName=livy-session-0>

<div class="alert alert-block alert-danger">
    WARNING: Do <strong>NOT</strong> remodify the below cell. It contains the function for loading data and all imports, and the function for running your code.
</div>

In [2]:
#export
from pyspark.sql.functions import col
from pyspark.sql import *

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
#### DO NOT CHANGE ANYTHING IN THIS CELL ####


def load_data():
    # Loads the data for this question. Do not change this function.
    # This function should only be called with the parameter 'small' or 'large'
    input_bucket = "s3://cse6242-ymyung3/team80"

    # Load Trip Data
    mean_path = '/yearly_variables_us_coordinates_mean_v1.csv'
    means = spark.read.csv(input_bucket + mean_path,
                           header=True,
                           inferSchema=True)
    print("Mean Count: ", means.count() )  

    # Load STD data
    std_path = '/yearly_variables_us_coordinates_std_v3.csv'    
    stds = spark.read.csv(input_bucket + std_path,
                            header=True,
                            inferSchema=True)
    print("Std Count: ", stds.count())
    
    # Load Other variables data
    other_path = '/yearly_variables_us_coordinates_others_v1.csv'    
    others = spark.read.csv(input_bucket + other_path,
                            header=True,
                            inferSchema=True)
    print("Other Count: ", others.count())    
    return means, stds, others


means, stds, others = load_data()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Mean Count:  546219
Std Count:  546219
Other Count:  546219

In [6]:
stds.createOrReplaceTempView("stds")
df = stds.sql_ctx.sql("select * from stds")
df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[_c0: int, index: int, latitude: double, longitude: double, elevation: double, state: string, std_air_temperature: double, std_alpha: double, std_aod: double, std_asymmetry: double, std_cld_opd_dcomp: double, std_cld_reff_dcomp: double, std_clearsky_dhi: double, std_clearsky_dni: double, std_clearsky_ghi: double, std_cloud_press_acha: double, std_cloud_type: double, std_dew_point: double, std_dhi: double, std_dni: double, std_ghi: double, std_ozone: double, std_relative_humidity: double, std_solar_zenith_angle: double, std_ssa: double, std_surface_albedo: double, std_surface_pressure: double, std_total_precipitable_water: double, std_wind_direction: double, std_wind_speed: double]

In [7]:
#export
def concat_three_df(df1, df2, df3):
    # Returns a Dataframe (mtrips) with Schema: DOLocationID, pcount

    df1.createOrReplaceTempView("means")
    df2.createOrReplaceTempView("stds")
    df3.createOrReplaceTempView("others")    

    df = df1.sql_ctx.sql(
'''
SELECT means.index, means.latitude, means.longitude,
means.elevation, means.state,
avg_air_temperature,
avg_alpha,
avg_aod,
avg_asymmetry,
avg_cld_opd_dcomp,
avg_cld_reff_dcomp,
avg_clearsky_dhi,
avg_clearsky_dni,
avg_clearsky_ghi,
avg_cloud_press_acha,
avg_cloud_type,
avg_dew_point,
avg_dhi,
avg_dni,
avg_ghi,
avg_ozone,
avg_relative_humidity,
avg_solar_zenith_angle,
avg_ssa,
avg_surface_albedo,
avg_surface_pressure,
avg_total_precipitable_water,
avg_wind_direction,
avg_wind_speed,
std_air_temperature,
std_alpha,
std_aod,
std_asymmetry,
std_cld_opd_dcomp,
std_cld_reff_dcomp,
std_clearsky_dhi,
std_clearsky_dni,
std_clearsky_ghi,
std_cloud_press_acha,
std_cloud_type,
std_dew_point,
std_dhi,
std_dni,
std_ghi,
std_ozone,
std_relative_humidity,
std_solar_zenith_angle,
std_ssa,
std_surface_albedo,
std_surface_pressure,
std_total_precipitable_water,
std_wind_direction,
std_wind_speed,
wind_u5,
wind_5_10,
wind_10_15,
wind_15_20,
wind_20_25,
wind_o25,
temp_u0,
temp_0_10,
temp_10_20,
temp_20_30,
temp_30_40,
temp_o40,
wind_power_density
FROM means INNER JOIN stds on means.index = stds.index INNER JOIN others ON means.index = others._c1 ORDER BY means.index
'''                                
    )
    return df

df = concat_three_df(means, stds, others)
df.show(5)
df.coalesce(1).write.option("header","true").csv("s3://cse6242-ymyung3/team80/result6.csv")
print('data saved into s3 bucket')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+--------+---------+---------+---------+-------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+-----------------+--------------------+------------------+------------------+-----------------+------------------+------------------+-------------------+---------------------+----------------------+------------------+--------------------+--------------------+----------------------------+------------------+------------------+-------------------+-------------------+--------------------+--------------------+-----------------+------------------+------------------+------------------+------------------+--------------------+------------------+------------------+------------------+-----------------+------------------+--------------------+---------------------+----------------------+--------------------+--------------------+--------------------+----------------------------+------------------+------